In [18]:
import sys
import os
import pandas as pd
# 현재 작업 디렉토리 기준으로 src 디렉토리 추가
notebook_dir = os.getcwd()  # 현재 Jupyter Notebook의 작업 디렉토리

sys.path.append(os.path.dirname(notebook_dir))


In [19]:
from src.tools.google_drive import ensure_data_files

# 데이터 파일 경로 가져오기
data_paths = ensure_data_files()
data_paths


기존 data가 존재합니다. 파일 경로를 반환합니다.


{'reviewer': '/Users/seongrok.kim/Github/yamyam-lab/data/reviewer.csv',
 'review_keyword': '/Users/seongrok.kim/Github/yamyam-lab/data/review_keyword.csv',
 'category': '/Users/seongrok.kim/Github/yamyam-lab/data/diner_category_raw.csv',
 'review': '/Users/seongrok.kim/Github/yamyam-lab/data/review.csv',
 'dinr_menu': '/Users/seongrok.kim/Github/yamyam-lab/data/dinr_menu.csv',
 'diner': '/Users/seongrok.kim/Github/yamyam-lab/data/diner.csv'}

In [20]:
# load data
diner = pd.read_csv(data_paths["diner"])
diner_category = pd.read_csv(data_paths["category"])

diner = pd.merge(diner, diner_category, on="diner_idx", how="left")


review = pd.read_csv(data_paths["review"])
reviewer = pd.read_csv(data_paths["reviewer"])

review = pd.merge(review, reviewer, on="reviewer_id", how="left")

/var/folders/dq/xrxyf7x57q504wf515d672p00000gn/T/ipykernel_71884/2461593576.py:2: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner = pd.read_csv(data_paths["diner"])


In [22]:
from src.constant.evaluation.qualitative import QualitativeReviewerId

test_reviewer_id = [QualitativeReviewerId.ROCKY, QualitativeReviewerId.HKY, QualitativeReviewerId.YEONSEO, QualitativeReviewerId.CHOCOCHO, QualitativeReviewerId.SONG, QualitativeReviewerId.PIG, QualitativeReviewerId.ICECREAM, QualitativeReviewerId.ALPHACA, QualitativeReviewerId.WOOK]

## 기존 평균 산출방법

In [29]:
import pandas as pd
import numpy as np


def create_target_column(review: pd.DataFrame) -> pd.DataFrame:
    """
    Create the target column for classification.
    """
    review = review.drop(columns=["reviewer_avg"])
    review["reviewer_avg"] = review.groupby("diner_idx")[
        "reviewer_review_score"
    ].transform("mean")

    review["target"] = (
        review["reviewer_review_score"] >= review["reviewer_avg"]
    ).astype(np.int8)

    return review
result_exited_avg = create_target_column(review)

# 첫 번째 리뷰어의 결과 상세 확인
first_reviewer = QualitativeReviewerId.ROCKY
first_reviewer_data = result_exited_avg[result_exited_avg['reviewer_id'] == first_reviewer].sort_values('reviewer_review_date')

print(f"\n=== 리뷰어 {first_reviewer}의 시점별 평균 변화 ===")
display_cols = ['reviewer_id', 'reviewer_review_score', 'reviewer_avg', 'target', 'reviewer_review_date']
available_display_cols = [col for col in display_cols if col in first_reviewer_data.columns]
print(first_reviewer_data[available_display_cols].to_string(index=False))


=== 리뷰어 893438059의 시점별 평균 변화 ===
 reviewer_id  reviewer_review_score  reviewer_avg  target reviewer_review_date
   893438059                    2.0      3.684211       0           2021-12-26
   893438059                    1.0      1.939633       0           2022-02-10
   893438059                    4.0      2.238095       1           2022-04-08
   893438059                    5.0      3.600000       1           2022-10-25
   893438059                    2.0      3.800000       0           2023-03-07
   893438059                    5.0      4.888889       1           2023-03-07
   893438059                    3.0      4.379310       0           2023-08-10
   893438059                    5.0      4.591837       1           2023-08-10
   893438059                    4.0      4.741361       0           2024-06-25
   893438059                    4.0      2.346154       1           2024-12-06
   893438059                    3.0      4.538462       0           2024-12-06
   893438059      

## 개선 평균 산출 방법

In [24]:
# 시점별 조정 모듈 import
from src.data.temporal_utils import (
    calculate_temporal_reviewer_avg, 
    estimate_temporal_badge_level,
    apply_temporal_adjustments,
    calculate_temporal_target
)

In [25]:
# 1. calculate_temporal_reviewer_avg 테스트
print("=== 1. 시점별 reviewer_avg 계산 테스트 ===")


# 시점별 reviewer_avg 계산
result_temporal_avg = calculate_temporal_reviewer_avg(review)

print("✓ calculate_temporal_reviewer_avg 함수 실행 완료")
print(f"결과 데이터 shape: {result_temporal_avg.shape}")

# 새로 추가된 컬럼 확인
new_cols = set(result_temporal_avg.columns) - set(review.columns)
print(f"새로 추가된 컬럼: {list(new_cols)}")

# 첫 번째 리뷰어의 결과 상세 확인
first_reviewer = QualitativeReviewerId.ROCKY
first_reviewer_data = result_temporal_avg[result_temporal_avg['reviewer_id'] == first_reviewer].sort_values('reviewer_review_date')

print(f"\n=== 리뷰어 {first_reviewer}의 시점별 평균 변화 ===")
display_cols = ['reviewer_id', 'reviewer_review_score', 'reviewer_avg', 'temporal_reviewer_avg', 'reviewer_review_date']
available_display_cols = [col for col in display_cols if col in first_reviewer_data.columns]
print(first_reviewer_data[available_display_cols].to_string(index=False))


=== 1. 시점별 reviewer_avg 계산 테스트 ===
✓ calculate_temporal_reviewer_avg 함수 실행 완료
결과 데이터 shape: (2287474, 13)
새로 추가된 컬럼: ['temporal_reviewer_avg']

=== 리뷰어 893438059의 시점별 평균 변화 ===
 reviewer_id  reviewer_review_score  reviewer_avg  temporal_reviewer_avg reviewer_review_date
   893438059                    2.0           3.8               2.000000           2021-12-26
   893438059                    1.0           3.8               2.000000           2022-02-10
   893438059                    4.0           3.8               1.500000           2022-04-08
   893438059                    5.0           3.8               2.333333           2022-10-25
   893438059                    2.0           3.8               3.000000           2023-03-07
   893438059                    5.0           3.8               2.800000           2023-03-07
   893438059                    3.0           3.8               3.166667           2023-08-10
   893438059                    5.0           3.8               3.14285

## ROCKY의 리뷰 히스토리로 본 두 평균 산출 비교

In [26]:
# 첫 번째 리뷰어의 결과 상세 확인
test_reviewer = QualitativeReviewerId.ROCKY
exited_method_reviewer_data = result_exited_avg[result_exited_avg['reviewer_id'] == test_reviewer].sort_values('reviewer_review_date')
temporal_method_reviewer_data = result_temporal_avg[result_temporal_avg['reviewer_id'] == test_reviewer].sort_values('reviewer_review_date')


# 두 DataFrame을 review_date 기준으로 merge
comparison_df = pd.merge(
    exited_method_reviewer_data[['review_id','reviewer_review_score', 'reviewer_review_date', 'reviewer_avg']],
    temporal_method_reviewer_data[['review_id','temporal_reviewer_avg']],
    on='review_id',
    how='inner',  # 공통된 날짜 기준으로 병합
    suffixes=('_exited', '_temporal')
)

# 보기 좋게 정렬
comparison_df = comparison_df.sort_values('reviewer_review_date').reset_index(drop=True)
comparison_df

,review_id,reviewer_review_score,reviewer_review_date,reviewer_avg,temporal_reviewer_avg
0,4793373,2.0,2021-12-26,3.685714,2.000000
1,5314585,4.0,2022-04-08,2.250000,1.500000
2,6448325,5.0,2022-10-25,3.625000,2.333333
3,7132384,5.0,2023-03-07,5.000000,2.800000
4,7132373,2.0,2023-03-07,3.923077,3.000000
5,8285685,3.0,2023-08-10,4.320000,3.166667
6,10424619,4.0,2024-06-25,4.726858,3.375000
7,11567104,3.0,2024-12-06,3.558052,3.583333
8,11567108,3.0,2024-12-06,4.636364,3.500000
9,11567093,5.0,2024-12-06,4.176471,3.454545


# badge_level

In [27]:
# 2. estimate_temporal_badge_level 테스트
print("=== 2. 시점별 badge_level 추정 테스트 ===")

# 시점별 badge_level 추정
result_temporal_badge = estimate_temporal_badge_level(result_temporal_avg)

print("✓ estimate_temporal_badge_level 함수 실행 완료")
print(f"결과 데이터 shape: {result_temporal_badge.shape}")

# 새로 추가된 컬럼 확인
new_cols_badge = set(result_temporal_badge.columns) - set(result_temporal_avg.columns)
print(f"새로 추가된 컬럼: {list(new_cols_badge)}")

# 첫 번째 리뷰어의 badge_level 변화 확인
first_reviewer_badge = result_temporal_badge[result_temporal_badge['reviewer_id'] == first_reviewer].sort_values('reviewer_review_date')

print(f"\n=== 리뷰어 {first_reviewer}의 시점별 badge_level 변화 ===")
badge_cols = ['reviewer_id', 'badge_level', 'temporal_badge_level', 'cumulative_review_count', 'review_progress_ratio', 'reviewer_review_date']
available_badge_cols = [col for col in badge_cols if col in first_reviewer_badge.columns]
print(first_reviewer_badge[available_badge_cols].to_string(index=False))


=== 2. 시점별 badge_level 추정 테스트 ===
✓ estimate_temporal_badge_level 함수 실행 완료
결과 데이터 shape: (2287474, 16)
새로 추가된 컬럼: ['cumulative_review_count', 'review_progress_ratio', 'temporal_badge_level']

=== 리뷰어 893438059의 시점별 badge_level 변화 ===
 reviewer_id  badge_level  temporal_badge_level  cumulative_review_count  review_progress_ratio reviewer_review_date
   893438059           23              1.210526                        1               0.052632           2021-12-26
   893438059           23              2.421053                        2               0.105263           2022-02-10
   893438059           23              3.631579                        3               0.157895           2022-04-08
   893438059           23              4.842105                        4               0.210526           2022-10-25
   893438059           23              6.052632                        5               0.263158           2023-03-07
   893438059           23              7.263158                 

In [30]:
# 3. 전체 시점별 조정 및 타겟 계산 테스트
print("=== 3. 전체 시점별 조정 및 타겟 계산 테스트 ===")

# apply_temporal_adjustments 함수로 한번에 모든 조정 적용
final_result = apply_temporal_adjustments(review.copy())

print("✓ apply_temporal_adjustments 함수 실행 완료")
print(f"최종 결과 데이터 shape: {final_result.shape}")

# 시점별 타겟 계산
final_result_with_target = calculate_temporal_target(final_result)

print("✓ calculate_temporal_target 함수 실행 완료")

# 기존 타겟과 시점별 타겟 비교
final_result_with_target['original_target'] = (
    final_result_with_target['reviewer_review_score'] >= final_result_with_target['reviewer_avg']
).astype(int)

print(f"\n=== 타겟 변화 분석 ===")
target_changes = final_result_with_target['original_target'] != final_result_with_target['temporal_target']
print(f"타겟이 변경된 리뷰 수: {target_changes.sum()}/{len(final_result_with_target)} ({target_changes.sum()/len(final_result_with_target)*100:.2f}%)")

# 첫 번째 리뷰어의 최종 결과 확인
first_reviewer_final = final_result_with_target[final_result_with_target['reviewer_id'] == first_reviewer].sort_values('reviewer_review_date')

print(f"\n=== 리뷰어 {first_reviewer}의 최종 결과 ===")
final_cols = [
    'reviewer_id', 'reviewer_review_score', 'reviewer_avg', 'temporal_reviewer_avg',
    'badge_level', 'temporal_badge_level', 'original_target', 'temporal_target', 'reviewer_review_date'
]
available_final_cols = [col for col in final_cols if col in first_reviewer_final.columns]
first_reviewer_final[available_final_cols]

=== 3. 전체 시점별 조정 및 타겟 계산 테스트 ===
✓ apply_temporal_adjustments 함수 실행 완료
최종 결과 데이터 shape: (2287474, 16)
✓ calculate_temporal_target 함수 실행 완료

=== 타겟 변화 분석 ===
타겟이 변경된 리뷰 수: 231649/2287474 (10.13%)

=== 리뷰어 893438059의 최종 결과 ===


,reviewer_id,reviewer_review_score,reviewer_avg,temporal_reviewer_avg,badge_level,temporal_badge_level,original_target,temporal_target,reviewer_review_date
814860,893438059,2.0,3.8,2.000000,23,1.210526,0,1,2021-12-26
814853,893438059,1.0,3.8,2.000000,23,2.421053,0,0,2022-02-10
814858,893438059,4.0,3.8,1.500000,23,3.631579,1,1,2022-04-08
814857,893438059,5.0,3.8,2.333333,23,4.842105,1,1,2022-10-25
814854,893438059,2.0,3.8,3.000000,23,6.052632,0,0,2023-03-07
814856,893438059,5.0,3.8,2.800000,23,7.263158,1,1,2023-03-07
814844,893438059,3.0,3.8,3.166667,23,8.473684,0,0,2023-08-10
814846,893438059,5.0,3.8,3.142857,23,9.684211,1,1,2023-08-10
814851,893438059,4.0,3.8,3.375000,23,10.894737,1,1,2024-06-25
814859,893438059,5.0,3.8,3.538462,23,16.947368,1,1,2024-12-06
